In [12]:
import numpy as np
import matplotlib.pyplot as plt 
from scipy.linalg import solve_banded
import pandas as pd
# We will try to solve for the function zeta using implict method and compare our answer to the one obtained through backward differenc
# Kriess Oliger Dissipation function has also been employed

def round_of_to_lowest_ten(x):
    return (x // 10)* 10 

def get_banded_form(matrix, bandwidth_up, bandwidth_down):
    n = matrix.shape[0]
    diagonals = np.zeros((bandwidth_up + bandwidth_down + 1, n))

    for i in range(-bandwidth_down, bandwidth_up + 1):
        for j in range(0, n - abs(i)):
            if i >= 0:
                diagonals[bandwidth_up - i, n - 1 -j] = np.diag(matrix, k = i)[n - abs(i) - 1 - j]
            else:
                diagonals[bandwidth_up - i, j] = np.diag(matrix, k = i)[j]
    return diagonals

def zeta_zero(p_given):
    m = 1
    Q = 0.9
    p = p_given
    A = 0.5
    b_couple = 200
    y_boundary =2* m 
    a = 0.1                       # Kreiss Oliger Dissipation coefficient
    h = 1/2**12                   # Step size
    n = 2**12 - 1                 # We only solve for the points which are not initially fixed

    # Definiing the implicit matrix
    M = np.zeros([n, n])

    # Defining the boundary parts of matrix 
    M[0, :5]         = [-25 + 12 * a * h * 35, 48 + 12 * a * h * (-104), -36 + 12 * a * h * 114, 16 + 12 * a * h * (-56), -3 + 12 * a * h * 11]
    M[1, :5]         = [-3 + 12 * a * h * 11, -10 + 12 * a * h * (-20), 18 + 12 * a * h * 6, -6 + 12 * a * h * 4, 1 + 12 * a * h * (-1)]
    M[n - 2, n - 4:] = [1 + 12 * a * h * (-1), -8 + 12 * a * h * 16, 0 + 12 * a * h * (-30), 8 + 12 * a * h * 16]
    M[n - 1, n - 4:] = [-1 + 12 * a * h * (-1), 6 + 12 * a * h * 4, -18 + 12 * a * h * 6, 10 + 12 * a * h * (-20)]
    
    # Bulk of the matrix
    for i in range(2, n - 2):       
        M[i][i - 2] = 1 + 12* a* h* (-1)
        M[i][i - 1] = -8 + 12* a* h* (16)
        M[i][i]     = 0 + 12* a* h* (-30) 
        M[i][i + 1] = 8 + 12* a* h* (16)
        M[i][i + 2] = -1 + 12* a* h* (-1)

    err = 10**(-3)
    Z = np.linspace(err, 1 - err, 2**(12))

    Z_mod = Z[:-1]      # Neglecting the final point
    b = np.zeros(n)
    
    for i in range(0, n):
            c1 = Q**2 / (m* Z_mod[i]**2)
            c2 = 4* A**2* m* Z_mod[i]**4 / (p**4 * (Z_mod[i] - 1)**6 )

            int1 = np.exp(- 4* Z_mod[i]**2 / (p**2* (Z_mod[i] - 1)**2 ))
            int2 = np.exp(- 2* Z_mod[i]**2 / (p**2* (Z_mod[i] - 1)**2 ))

            term1 = c1* np.exp( - A**4* b_couple* int1 ) 
            term2 = c2* int2
            term = term1 + term2    
            b[i] = term* 12* h

    
    # Fixing initial conditions
    b[n - 1] = b[n - 1] - ((3 + 11* 12*a*h)* y_boundary) 
    b[n - 2] = b[n - 2] - ((-1 - 1* 12*a*h)* y_boundary) 


    # Using banded technique
    banded_M = get_banded_form(M, 4, 3)
    Y = solve_banded((3, 4), banded_M, b)
    
    Y_mod = np.append(Y, y_boundary)
    R = m* Z/(1 - Z)

    cut_off = round_of_to_lowest_ten(np.argmin(abs(Y_mod - R)))

    print(f" The position of initial apparant horizon is r = {R[cut_off]} \n z = {Z[cut_off]} \n m = {cut_off}") 
    
    m = n - cut_off + 1
    s = np.sqrt(Y_mod[cut_off:])
    r = R[cut_off:]
    z = Z[cut_off:]

    return m, z, r, s

# zeta_zero(np.float128(2.116895853824))


In [13]:
from numba import jit
@jit(nopython = True)

def get_banded_form(matrix, bandwidth_up, bandwidth_down):
    n = matrix.shape[0]
    diagonals = np.zeros((bandwidth_up + bandwidth_down + 1, n))

    for i in range(-bandwidth_down, bandwidth_up + 1):
        for j in range(0, n - abs(i)):
            if i >= 0:
                diagonals[bandwidth_up - i, n - 1 -j] = np.diag(matrix, k = i)[n - abs(i) - 1 - j]
            else:
                diagonals[bandwidth_up - i, j] = np.diag(matrix, k = i)[j]
    return diagonals

def Updated_implicit_Alpha(array_size, step_size, d_dz, r, Y):
    n = array_size - 1
    h = step_size
    a = 0.1  # Dissipation coefficient for alpha
 
    # l_ini = np.float128(0)

    # We will be using the implicit method to solve this equation
    # Initializing the Right-hand side of the equation
    b = -12 * h * r[:-1] * np.sqrt(r[:-1]) * Y[2][:-1] * d_dz(Y[0][:-1]) / Y[1][:-1]

    # Defining the matrix
    M = np.zeros([n, n])

    # Defining the boundary parts of the matrix
    M[0, :5] = [-25 + 12 * a * h * 35, 48 + 12 * a * h * (-104), -36 + 12 * a * h * 114, 16 + 12 * a * h * (-56), -3 + 12 * a * h * 11]
    M[1, :5] = [-3 + 12 * a * h * 11, -10 + 12 * a * h * (-20), 18 + 12 * a * h * 6, -6 + 12 * a * h * 4, 1 + 12 * a * h * (-1)]
    M[n - 2, n - 4:] = [1 + 12 * a * h * (-1), -8 + 12 * a * h * 16, 0 + 12 * a * h * (-30), 8 + 12 * a * h * 16]
    M[n - 1, n - 4:] = [-1 + 12 * a * h * (-1), 6 + 12 * a * h * 4, -18 + 12 * a * h * 6, 10 + 12 * a * h * (-20)]

    # Bulk of the matrix
    for i in range(2, n - 2):
        M[i][i - 2] = 1 + 12 * a * h * (-1)
        M[i][i - 1] = -8 + 12 * a * h * (16)
        M[i][i] = 0 + 12 * a * h * (-30)
        M[i][i + 1] = 8 + 12 * a * h * (16)
        M[i][i + 2] = -1 + 12 * a * h * (-1)

    banded_M = get_banded_form(M, 4, 3)
    l_int = solve_banded((3, 4), banded_M, b)

    # Adding back the final value
    l = np.append(l_int, 0)

    # Returning alpha = e^l
    return np.exp(l)


In [14]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
from findiff import FinDiff

def evol(p):
    M = 1
    Q = 0.9
    b = 200
    A = 0.5

    # Calling the implicit s function to get our initial data. We will be working with s0 instead of s because of the decay issues

    m, z, r, s0 = zeta_zero(p)

    # Defining Alpha at t = 0
    alpha0 = np.ones(m)

    # Defining the alpha list as it's not part of the evolving ODEs and just a constraint at every point in time.
    # Hence has to be manually updated, which the list definition makes it easier to do so
    alpha = [alpha0]

    # Defining Pi at t = 0
    pi0 = np.zeros(m)

    # Defining Phi in the required range at t = 0
    phi0 = A * np.exp(-z ** 2 / (p ** 2 * (1 - z) ** 2))

    # Defining the derivative operator from findiff
    dz = z[1] - z[0]
    d_dz = FinDiff(0, dz, 1, acc=4)


    # Defining the time interval
    t_ini = 0
    t_final = 300
    N = 300000  # No of grid points in time
    dt = (t_final - t_ini) / N
    n_sys = 3  # No of simultaneous system of equations we are trying to evolve

    # Defining the RHS of the differential equations

    def phi(t, Y):
        Cap_phi = d_dz(Y[0]) / d_dz(r)
        return alpha[-1] * (Y[2] + Cap_phi * Y[1] / np.sqrt(r))

    def s(t, Y):
        Cap_phi = d_dz(Y[0]) / d_dz(r)
        return (r ** 2 * alpha[-1] / Y[1]) * (Y[2] + Cap_phi * Y[1] / np.sqrt(r)) * (
                    Y[2] * Y[1] / np.sqrt(r) + Cap_phi)

    def Pi(t, Y):
        Cap_phi = d_dz(Y[0]) / d_dz(r)
        term1 = d_dz((Y[2] * Y[1] / np.sqrt(r) + Cap_phi) * alpha[-1] * r ** 2) / d_dz(r)
        term2 = alpha[-1] * Q ** 2 * 4 * b * np.exp(-b * Y[0] ** 4) * Y[0] ** 3 / (2 * r ** 4)
        return term1 / r ** 2 + term2

    # Defining the system of ODE to be easier to iterate over
    System = [phi, s, Pi]

    # Defining the time-grid points
    T = np.linspace(t_ini, t_final, N)

    # Setting the initial Conditions at t = 0 in the Funcs array
    Funcs_i = np.array([phi0, s0, pi0])

    # Initializing the Runge-Kutta k-values
    k1 = np.zeros([n_sys, m])
    k2 = np.zeros([n_sys, m])
    k3 = np.zeros([n_sys, m])
    k4 = np.zeros([n_sys, m])

    apparant_horizon_phi = np.zeros(N)
    apparant_horizon_radius = np.zeros(N)
    ones = np.ones(m)

    # Encoding the position at t = 0
    index = np.argmin(abs(Funcs_i[1]/ np.sqrt(r) - ones))
    apparant_horizon_phi[0] = phi0[index]
    apparant_horizon_radius[0] = r[index]

    # Starting the time-loop. "i" - index is for the time and "j" - index is for the different equations
    for i in range(0, N - 1):
        
        for j in range(0, n_sys):
            k1[j] = dt * (System[j](T[i], Funcs_i))

        for j in range(0, n_sys):
            k2[j] = dt * (System[j](T[i] + dt / 2, Funcs_i + k1 / 2))

        for j in range(0, n_sys):
            k3[j] = dt * (System[j](T[i] + dt / 2, Funcs_i + k2 / 2))

        for j in range(0, n_sys):
            k4[j] = dt * (System[j](T[i] + dt, Funcs_i + k3))

        Funcs_i = Funcs_i + (k1 + 2 * k2 + 2 * k3 + k4) / 6

        print(i)
        alpha.append(Updated_implicit_Alpha(m, dz, d_dz, r, Funcs_i))

        index = np.argmin(abs(Funcs_i[1]/ np.sqrt(r) - ones))
        apparant_horizon_phi[i + 1] = Funcs_i[0][index]
        apparant_horizon_radius[i + 1] = r[index]

    return apparant_horizon_phi, apparant_horizon_radius, T


In [ ]:
import matplotlib.pyplot as plt

p1, r1, t1 =  evol(2.113772888204 + 10**(-11))

df = pd.DataFrame({'Column1': p1,'Column2': r1})
df.to_csv('s4.txt' , sep = '\t' , index= False, header = False)

plt.plot(t1, p1)
plt.xlabel("time")
plt.ylabel("Phi_h")
plt.show()

plt.plot(t1, r1)
plt.xlabel("time")
plt.ylabel("r_h")
plt.show()


In [ ]:
dt = 0.001
d_dt = FinDiff(0, dt, 1, acc = 6)
dp1_dt = d_dt(p1)
plt.plot(t1,1.2 + 2*np.log(abs(dp1_dt)))
plt.show()

In [ ]:

from matplotlib.animation import FuncAnimation

N = 2**(12)
err = 10**(-3)
Z = np.linspace(err, 1 - err, N)
z = Z[2320:]
r = z / (1 - z)

T = 300
dt = 0.001
# Set up the plot
fig = plt.figure()
ax = plt.gca()
ax.plot(funcs1[0, 0, :])

# Setting frequency
k = 500
def update(i):
    ax.cla()
    ax.plot(z, funcs1[k* i, 0, :])
    ax.set_title('Time = {:.3f}'.format(i*dt*k))
    ax.set_ylim(min(funcs1[0, 0, :]), max(funcs1[0, 0, :]))
    plt.xlabel("Compactified z")
    plt.ylabel("Phi")
    return None

no_frames = int(T/(dt* k))

# Initialize the animation
ani = FuncAnimation(fig, update, frames=no_frames, interval=10)
ani.save('scalar-field.gif', writer='pillow', fps=10)
plt.show()

